In [ ]:
!pip install transformers
!pip install torch


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the tokenizer and the model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)  # 3 labels for NLI (entailment, neutral, contradiction)

# Ensure the model is in evaluation mode
model.eval()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [13]:
def prepare_data(premise, hypothesis):
    # Tokenize the inputs
    inputs = tokenizer.encode_plus(premise, hypothesis, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
    return inputs
premise = "The man is eating apple."
hypothesis = "The man is eating apple."
inputs = prepare_data(premise, hypothesis)


In [14]:
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

# Mapping the predicted class to a label
label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
prediction = label_map[predicted_class]

print(f"Prediction: {prediction}")


Prediction: contradiction


In [15]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the tokenizer and the NLI model
tokenizer = BertTokenizer.from_pretrained('textattack/bert-base-uncased-snli')
model = BertForSequenceClassification.from_pretrained('textattack/bert-base-uncased-snli')

# Ensure the model is in evaluation mode
model.eval()

def prepare_data(premise, hypothesis):
    # Tokenize the inputs
    inputs = tokenizer.encode_plus(premise, hypothesis, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
    return inputs

def predict(premise, hypothesis):
    inputs = prepare_data(premise, hypothesis)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
    # Mapping the predicted class to a label
    label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
    return label_map[predicted_class]

# Test examples
premise1 = "A man inspects the uniform of a figure in some East Asian country."
hypothesis1 = "The man is sleeping."
prediction1 = predict(premise1, hypothesis1)
print(f"Premise: {premise1}")
print(f"Hypothesis: {hypothesis1}")
print(f"Prediction: {prediction1}")

premise2 = "A soccer game with multiple males playing."
hypothesis2 = "Some men are playing a sport."
prediction2 = predict(premise2, hypothesis2)
print(f"Premise: {premise2}")
print(f"Hypothesis: {hypothesis2}")
print(f"Prediction: {prediction2}")

premise3 = "An older and younger man smiling."
hypothesis3 = "Two men are smiling and laughing at the cats playing on the floor."
prediction3 = predict(premise3, hypothesis3)
print(f"Premise: {premise3}")
print(f"Hypothesis: {hypothesis3}")
print(f"Prediction: {prediction3}")


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/630 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Premise: A man inspects the uniform of a figure in some East Asian country.
Hypothesis: The man is sleeping.
Prediction: entailment
Premise: A soccer game with multiple males playing.
Hypothesis: Some men are playing a sport.
Prediction: neutral
Premise: An older and younger man smiling.
Hypothesis: Two men are smiling and laughing at the cats playing on the floor.
Prediction: contradiction
